In [2]:
from keras.datasets import fashion_mnist
((X_train, Y_train),(X_test, Y_test)) = fashion_mnist.load_data()

Using TensorFlow backend.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [4]:
import pandas as pd
y_train = pd.get_dummies(Y_train.astype(str))
y_test = pd.get_dummies(Y_test.astype(str))
y_train = y_train.values
y_test = y_test.values

In [5]:
# Initialize Weights

def init_weights(shape):
    initializer = tf.variance_scaling_initializer()
    return tf.Variable(initializer(shape), dtype=tf.float32)

In [6]:
# Initialize Bias

def init_bias (shape):
    return tf.Variable(tf.zeros(shape))

In [7]:
# Convolution 2D

def conv2d(x, W):
    # x --> input tensor [batch, H, W, channels]
    # W --> Kernel [Filter H, FilterW, Channel In, Channel out]
    
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

In [8]:
# Pooling

def max_pool_2by2(x):
    # x --> input tensor [batch, H, W, channels]
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [9]:
# Convolutional Layer

def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [10]:
# Normal Layer

def normal_full_layer (input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer,W)+b

In [11]:
# Placeholder

x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

hold_prob = tf.placeholder(tf.float32)

In [12]:
x_train = X_train.reshape(60000, 784)/255
x_test = X_test.reshape(10000, 784)/255

In [13]:
x_image = tf.reshape(x,[-1,28,28,1])

In [14]:
convo_1 = convolutional_layer(x_image, shape=[5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

convo_2 = convolutional_layer(convo_1_pooling, shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

convo_2_flat = tf.reshape(convo_2_pooling, [-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

# drop out
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

y_pred = normal_full_layer(full_one_dropout, 10)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [29]:
# Loss function

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [30]:
# Optimizer

optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [31]:
init = tf.global_variables_initializer()

In [35]:
steps = 50
batch_size = 100
num_batches = len(x_train)//batch_size
with tf.Session() as sess:
        sess.run(init)
        
        for i in range(steps):
            
            x_batch = np.array_split(x_train, num_batches)
            y_batch = np.array_split(y_train, num_batches)
            
            for j in range(num_batches):
                
                batch_x = x_batch[j]
                batch_y = y_batch[j]
                
                sess.run(train, feed_dict={x:batch_x, y_true:batch_y, hold_prob:0.5})
            
            if i%10==0:
                
                print('ON Step: {}'.format(i))
                print('ACC: ')
                matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
                acc = tf.reduce_mean(tf.cast(matches, tf.float32))
                
                print(sess.run(acc, feed_dict={x:x_test, y_true:y_test,hold_prob:1.0}))
                
                print('\n')

ON Step: 0
ACC: 
0.8861


ON Step: 10
ACC: 
0.9209


ON Step: 20
ACC: 
0.919


ON Step: 30
ACC: 
0.9121


ON Step: 40
ACC: 
0.918


